# Practice round: Chinese-English translation

Huggingface transformer doc: https://huggingface.co/transformers/

Huggingface tokenizer doc: https://huggingface.co/transformers/

Useful resources from huggingface -- fine-tuning a model from scratch: https://huggingface.co/blog/how-to-train

The code I wrote before might be helpful: https://github.com/submal/ctec-lambus/blob/master/xprmt/xprmt_06.ipynb

A code example of fine-tuning T5 for text summarization: https://towardsdatascience.com/fine-tuning-a-t5-transformer-for-any-summarization-task-82334c64c81 

In [14]:
import json
import pandas as pd
import jieba
from tokenizers import SentencePieceBPETokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Model, AdamW

# Enable GPU if possible 
device = torch.device(
    'cuda:0' if torch.cuda.is_available() else 'cpu'
)
print(f'device = {device}')

device = cuda:0


## Load data

In [2]:
with open('./cn_en_weibo_data/data.cn-en.json', 'r', encoding = 'utf-8') as myfile:
    raw = myfile.read().split('\n')  

# Turn raw strings into a list of dictionaries
weiboDict = [json.loads(line) for line in raw]

weiboDf = pd.DataFrame(weiboDict)

weiboDf.tail()

,id,source,target
1998,3477898759956095,能发现自己的错误是智慧，能改正自己的错误是勇敢。喜欢请关注,"Can find their own mistakes is wisdom, to corr..."
1999,3526354320111701,我的死后日願望是，可以有世界末日啦！@browNsugaR 我在:,Very good music in Suns [good] //@browNsugaR:E...
2000,3558553027867596,年后，让你觉得更失望的不是你做过的事情，而是你没有做过的事情。所以，一直想做的事情，不要再拖了,online#Twenty years from now you will be more ...
2001,3482173493644453,说：“在小寨军区侧门，有人维权。过来个军车就给人家看。不过貌似四医大不归省军区管吧,"It's hard for her to win,she should explicit m..."
2002,3562778084139688,没有受伤，不懂坚强；不犯错误，难以成长；未曾失败，何来成功。,Day78：You'll never be brave if you don't get h...


It seems the data is far from clean. However, for prototyping purpose, we will not focus too much on cleaning right now. 

## Parsing and tokenizing Chinese texts

We use `jieba` library (结巴分词) for parsing Chinese text. For more information, see https://github.com/fxsjy/jieba/

In [3]:
chTexts = weiboDf['source']
enTexts = weiboDf['target']

# Tokenize all Chinese texts in the dataframe and store as a list
chTokensGen = [jieba.cut(sentence) for sentence in chTexts]

# Output a sample tokenization
print(list(chTokensGen[0]))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\presu\AppData\Local\Temp\jieba.cache
Loading model cost 0.691 seconds.
Prefix dict has been built successfully.


['好', '的', '爱情', '使', '你', '通过', '一个', '人', '看到', '整个', '世界', '，', '坏', '的', '爱情', '使', '你', '为了', '一个', '人', '舍弃', '整个', '世界', '。']


It turns out with tokenizers based on `sentencePiece`, the tokenization happens at sentence level, and the tokenizer is trained recognize subwords. Therefore we will not use other parsers for now. 

In [4]:
pathAllCh = './cn_en_weibo_data/allCh.txt'
pathAllEn = './cn_en_weibo_data/allEn.txt'

# Store all Chinese text in a single file 
with open(pathAllCh, 'w', encoding = 'utf-8') as file: 
    for line in chTexts:
        file.write(line + '\n')
    file.close()
    
# Store all English text in a single file 
with open(pathAllEn, 'w', encoding = 'utf-8') as file: 
    for line in enTexts: 
        file.write(line + '\n')
    file.close()

My feeling is that we cannot use a pretrained tokenizer to train it from scratch. Instead, we might need to import Byte-Pair Encoding, or WordPiece, or SentencePiece by scratch. 

https://huggingface.co/transformers/tokenizer_summary.html#sentencepiece

https://github.com/huggingface/tokenizers

In the following cell, we train a `SentencePiece` tokenizer. 

In [12]:
chTokenizer = SentencePieceBPETokenizer()

chTokenizer.train([pathAllCh], 
                vocab_size = 20000, 
                special_tokens = ['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

# Show an example of tokenizer works
output = chTokenizer.encode(chTexts[0])
print(output.ids, output.tokens, output.offsets, output.attention_mask)

# We shall save the tokenizer to disk 
chTokenizer.save_model('.', 'myTokenizer')

[6941, 1490, 2253, 1036, 1158, 2195, 3013, 339, 2037, 1490, 1197, 1036, 5754, 4099, 94] ['▁好的爱情', '使你', '通过', '一个人', '看到', '整个', '世界,', '坏', '的爱情', '使你', '为了', '一个人', '舍弃', '整个世界', '。'] [(0, 4), (4, 6), (6, 8), (8, 11), (11, 13), (13, 15), (15, 18), (18, 19), (19, 22), (22, 24), (24, 26), (26, 29), (29, 31), (31, 35), (35, 36)] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


['.\\myTokenizer-vocab.json', '.\\myTokenizer-merges.txt']

We also have the option to encode a list of texts as a batch. 

In [30]:
output_batch = chTokenizer.encode_batch(chTexts[:3])

for output in output_batch:
    print(output.tokens)

['▁好的爱情', '使你', '通过', '一个人', '看到', '整个', '世界,', '坏', '的爱情', '使你', '为了', '一个人', '舍弃', '整个世界', '。']
['▁如果你', '真', '要梦想', '成真', ',就', '先', '从梦中', '醒来', '。']
['▁有时候,', '最好的', '方法就是', '什么', '也不', '做']
None


## Demo: padding and truncation

Huggingface tokenizer allows us to pad or truncate according to a length. The following are common utilities for padding and truncation: 

`Tokenizer.enable_padding(**args)` -- Enable padding

`Tokenizer.padding` -- Info about padding

`Tokenizer.no_padding()` -- Disable padding

`Tokenizer.enable_truncation(**args)` -- Enable truncation 

`Tokenizer.truncation` -- Info about truncation 

`Tokenizer.no_truncation()` -- Disable truncation 

In [32]:
# With Padding
chTokenizer.enable_padding(length = 15)

output_batch = chTokenizer.encode_batch(chTexts[:3])

for output in output_batch:
    print(output.tokens)
    
print(chTokenizer.padding)

['▁好的爱情', '使你', '通过', '一个人', '看到', '整个', '世界,', '坏', '的爱情', '使你', '为了', '一个人', '舍弃', '整个世界', '。']
['▁如果你', '真', '要梦想', '成真', ',就', '先', '从梦中', '醒来', '。', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['▁有时候,', '最好的', '方法就是', '什么', '也不', '做', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
{'length': 15, 'pad_to_multiple_of': None, 'pad_id': 0, 'pad_token': '[PAD]', 'pad_type_id': 0, 'direction': 'right'}


In [34]:
# No padding
chTokenizer.no_padding()

output_batch = chTokenizer.encode_batch(chTexts[:3])

for output in output_batch:
    print(output.tokens)
    
print(chTokenizer.padding)

['▁好的爱情', '使你', '通过', '一个人', '看到', '整个', '世界,', '坏', '的爱情', '使你', '为了', '一个人', '舍弃', '整个世界', '。']
['▁如果你', '真', '要梦想', '成真', ',就', '先', '从梦中', '醒来', '。']
['▁有时候,', '最好的', '方法就是', '什么', '也不', '做']
None


In [37]:
# With truncation
chTokenizer.enable_truncation(max_length = 3)

output_batch = chTokenizer.encode_batch(chTexts[:3])

for output in output_batch:
    print(output.tokens)
    
print(chTokenizer.truncation)

['▁好的爱情', '使你', '通过']
['▁如果你', '真', '要梦想']
['▁有时候,', '最好的', '方法就是']
{'max_length': 3, 'stride': 0, 'strategy': 'longest_first'}


In [38]:
# No truncation
chTokenizer.no_truncation()

output_batch = chTokenizer.encode_batch(chTexts[:3])

for output in output_batch:
    print(output.tokens)
    
print(chTokenizer.truncation)

['▁好的爱情', '使你', '通过', '一个人', '看到', '整个', '世界,', '坏', '的爱情', '使你', '为了', '一个人', '舍弃', '整个世界', '。']
['▁如果你', '真', '要梦想', '成真', ',就', '先', '从梦中', '醒来', '。']
['▁有时候,', '最好的', '方法就是', '什么', '也不', '做']
None


<span style="color:red;">Pending problem.</span> As I tried to follow the tutorial and load the tokenizer saved on disk, unexpected error was reported. For now, skip loading saved tokenizer and proceed with other important steps.  

<span style="color:red;">Bottleneck for now.</span> Do we need special token for T5? If yes, how to insert special T5 tokens into our tokenization? Similar to `tokenizers.processors.BertProcessing`, do we have `tokenizers.processors.T5Processing`? 

<span style="color:red;">Solution.</span> 1. Thoroughtly read documentation for T5 model in huggingface doc; 2. Explore `huggingface/tokenizers` library on github. 

For now, halt with tokenizer and proceed with language model until bumping into problems. Keep in mind the confusion about special token. 

## Tokenizing English text

In [13]:
enTokenizer = SentencePieceBPETokenizer()

enTokenizer.train([pathAllEn], 
                vocab_size = 20000, 
               special_tokens = ['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

# Show an example of tokenizer works
output = enTokenizer.encode(enTexts[0])
print(output.ids, output.tokens, output.offsets, output.attention_mask)

# We shall save the tokenizer to disk 
# tokenizer.save_model('.', 'myTokenizer')

[2101, 961, 1407, 874, 1145, 875, 1771, 1080, 1103, 1016, 1140, 1581, 1459, 961, 1407, 874, 3558, 875, 1771, 1080, 925, 1016, 1140, 20] ['▁Good', '▁love', '▁makes', '▁you', '▁see', '▁the', '▁whole', '▁world', '▁from', '▁one', '▁person', '▁while', '▁bad', '▁love', '▁makes', '▁you', '▁abandon', '▁the', '▁whole', '▁world', '▁for', '▁one', '▁person', '.'] [(0, 4), (4, 9), (9, 15), (15, 19), (19, 23), (23, 27), (27, 33), (33, 39), (39, 44), (44, 48), (48, 55), (55, 61), (61, 65), (65, 70), (70, 76), (76, 80), (80, 88), (88, 92), (92, 98), (98, 104), (104, 108), (108, 112), (112, 119), (119, 120)] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Preprocess before training

To utilize PyTorch and GPU computation, we need to create instances of `Dataset` object. 

`DataLoader` class allows us to iterate a dataset with given batch size. 

The following cell overwrites `Dataset` class. 

In [44]:
class MyDataset(Dataset):
    
    '''
    Load original data and util from memory or file
    Texts must be passed as lists 
    '''
    def __init__(self, 
                 chTexts, enTexts, # Suppose the two colums are of the same length
                 chTokenizer, enTokenizer, 
                 chMaxLen, enMaxLen): 
        super().__init__()
        self.chTexts = chTexts 
        self.enTexts = enTexts
        self.chTokenizer = chTokenizer
        self.enTokenizer = enTokenizer 
        
        # Enable padding and truncation
        self.chTokenizer.enable_padding(length = chMaxLen)
        self.chTokenizer.enable_truncation(max_length = chMaxLen)
        self.enTokenizer.enable_padding(length = enMaxLen)
        self.enTokenizer.enable_truncation(max_length = enMaxLen)
        
    '''
    Return the size of dataset
    '''
    def __len__(self):
        return len(self.chTexts)
    
    '''
    -- The routine for querying one data entry 
    -- The index of must be specified as an argument
    -- Return a dictionary 
    '''
    def __getitem__(self, idx): 
        # Apply tokenizer 
        chOutputs = chTokenizer.encode(chTexts[idx])
        enOutputs = enTokenizer.encode(enTexts[idx])
        
        # Get numerical tokens
        chEncoding = chOutputs.ids
        enEncoding = enOutputs.ids
        
        # Get attention mask 
        chAttention = chOutputs.attention_mask
        enAttention = enOutputs.attention_mask
        
        return {
            'ch': {
                'ids': torch.tensor(chEncoding).to(device), 
                'attention_mask': torch.tensor(chAttention).to(device)
            }, 
            'en': {
                'ids': torch.tensor(enEncoding).to(device), 
                'attention_mask': torch.tensor(enAttention).to(device)
            } 
        }

Instantiate `Dataset` and `DataLoader`

In [45]:
MAX_LEN = 70

dataset = MyDataset(chTexts, enTexts, 
                    chTokenizer, enTokenizer, 
                    chMaxLen = MAX_LEN, enMaxLen = MAX_LEN)

dataloader = DataLoader(dataset, batch_size = 16, num_workers = 0)

In [54]:
# Verify the tokenizer is working

output = dataset.chTokenizer.encode('我在这')

print(output.ids, output.tokens, output.attention_mask)

[2260, 915, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ['▁我在', '这', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'] [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Training

In [10]:
# Load T5 model
model = T5Model.from_pretrained(
    't5-small', return_dict = True
).to(device)

# Set to training mode
model.train(); 

Some weights of T5Model were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We can choose to either use PyTorch native or `Trainer` class in huggingface to fine-tune the model. For now we use PyTorch native. 

In [17]:
optimizer = AdamW(model.parameters(), lr = 1e-5)